In [22]:
import numpy as np
import pandas as pd
#from sklearn.cluster import KMeans
import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial

In [23]:
data = pd.read_csv(r"C:\Users\Dell\Downloads\data.csv")
labels = pd.read_csv(r"C:\Users\Dell\Downloads\label.csv",names=['label'],header=None)

In [24]:
data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
data.count()

0        9999
0.1      9999
0.2      9999
0.3      9999
0.4      9999
         ... 
0.663    9999
0.664    9999
0.665    9999
0.666    9999
0.667    9999
Length: 784, dtype: int64

In [26]:
#Split data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split( data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( labels, test_size=0.08, random_state=42)

In [27]:
class KMeans:
    
    def calculate_SSE(self, centroid_value_dict, centroid_dict,data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0
            # np.sum()
            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def Initialize_Centroids(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jaccard_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        #Mode = 1 => Euclidean np.linalg.norm(x-list(data.iloc[i,:]))
        #Mode = 2 => Jaccard
        #Mode = 3 => Cosine
        centroid_value_dict = self.Initialize_Centroids(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centroid_value_dict,centroid_dict

def predict_cluster_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

def jaccard_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [28]:
model1 = KMeans()
centroids1,clusters1 = model1.train_Kmeans(data,10, max_iter=60,mode=1)

Tolerance for the Iteration  0 :  32254.23245614035
Tolerance for the Iteration  1 :  8296.586141835547
Tolerance for the Iteration  2 :  3260.818601851155
Tolerance for the Iteration  3 :  2025.034203807486
Tolerance for the Iteration  4 :  1732.8522348401511
Tolerance for the Iteration  5 :  1963.7397721250659
Tolerance for the Iteration  6 :  2041.3231403609047
Tolerance for the Iteration  7 :  1829.297414004067
Tolerance for the Iteration  8 :  1574.0588610702014
Tolerance for the Iteration  9 :  1220.5732089170385
Tolerance for the Iteration  10 :  1538.8432524067746
Tolerance for the Iteration  11 :  1841.203560649075
Tolerance for the Iteration  12 :  2025.0568850478091
Tolerance for the Iteration  13 :  1853.022536011841
Tolerance for the Iteration  14 :  1304.584630040761
Tolerance for the Iteration  15 :  942.8894695046376
Tolerance for the Iteration  16 :  701.4182959166651
Tolerance for the Iteration  17 :  402.47550564749145
Tolerance for the Iteration  18 :  278.279715620

In [29]:
Euclidean_SSE = model1.calculate_SSE(centroids1,clusters1,data)

In [30]:
print("Euclidean SSE:",Euclidean_SSE)

Euclidean SSE: 25400857935.882473


In [31]:
cluster_labels1 = predict_cluster_labels(centroids1,clusters1,labels)
cluster_labels1

array([5, 6, 7, 0, 1, 2, 9, 3, 0, 8])

In [32]:
Accuracy_Euclidean = accuracy(centroids1, cluster_labels1,test_data,test_labels)
Accuracy_Euclidean

0.07625

In [33]:
model2 = KMeans()
centroids2,clusters2 = model2.train_Kmeans(data,10, max_iter=60,mode=2)
Jaccard_SSE = model2.calculate_SSE(centroids2,clusters2,data)

Tolerance for the Iteration  0 :  40359.3950617284
Tolerance for the Iteration  1 :  3469.509275145989
Tolerance for the Iteration  2 :  2290.128650890556
Tolerance for the Iteration  3 :  1214.6573845087632
Tolerance for the Iteration  4 :  1808.618239601738
Tolerance for the Iteration  5 :  891.6232377427688
Tolerance for the Iteration  6 :  2383.0804751063342
Tolerance for the Iteration  7 :  1007.4817473639257
Tolerance for the Iteration  8 :  1576.5527567992258
Tolerance for the Iteration  9 :  1056.6711059994889
Tolerance for the Iteration  10 :  1152.0593295702215
Tolerance for the Iteration  11 :  1769.8232279749714
Tolerance for the Iteration  12 :  626.3967118532889
Tolerance for the Iteration  13 :  435.9521602485795
Tolerance for the Iteration  14 :  0.0


In [35]:
print("Jacard SSE:",Jaccard_SSE)

Jacard SSE: 34361687572.938736


In [36]:
cluster_labels2 = predict_cluster_labels(centroids2,clusters2,labels)
cluster_labels2

array([1, 3, 6, 6, 8, 4, 3, 0, 4, 8])

In [37]:
Accuracy_Jaccard = accuracy(centroids2, cluster_labels2,test_data,test_labels)
Accuracy_Jaccard

0.1075

In [38]:
model3 = KMeans()
centroids3,clusters3 = model3.train_Kmeans(data,10, max_iter = 60,mode=3)

Tolerance for the Iteration  0 :  28833.31961591221
Tolerance for the Iteration  1 :  5549.168226548903
Tolerance for the Iteration  2 :  3372.9536897844123
Tolerance for the Iteration  3 :  1991.881215510813
Tolerance for the Iteration  4 :  2797.1746134577993
Tolerance for the Iteration  5 :  3210.32656870275
Tolerance for the Iteration  6 :  3254.535539817598
Tolerance for the Iteration  7 :  2098.911349818512
Tolerance for the Iteration  8 :  1453.2547841209243
Tolerance for the Iteration  9 :  850.3227117701961
Tolerance for the Iteration  10 :  535.9610758885685
Tolerance for the Iteration  11 :  375.6998356115431
Tolerance for the Iteration  12 :  311.52529039892863
Tolerance for the Iteration  13 :  287.2111850045258
Tolerance for the Iteration  14 :  342.5249741200827
Tolerance for the Iteration  15 :  346.0405913317767
Tolerance for the Iteration  16 :  592.3488266796495
Tolerance for the Iteration  17 :  628.8413423602193
Tolerance for the Iteration  18 :  630.6236768150957


In [39]:
Cosine_SSE = model3.calculate_SSE(centroids3,clusters3,data)

In [40]:
print("Euclidean SSE:",Euclidean_SSE)
print("Jacard SSE:",Jaccard_SSE)
print("Cosine SSE :",Cosine_SSE)

Euclidean SSE: 25400857935.882473
Jacard SSE: 34361687572.938736
Cosine SSE : 25609570725.782463


In [41]:
cluster_labels3 = predict_cluster_labels(centroids3,clusters3,labels)
cluster_labels3

array([2, 7, 3, 0, 3, 1, 8, 0, 9, 6])

In [42]:
Accuracy_Cosine = accuracy(centroids3, cluster_labels3,test_data,test_labels)
Accuracy_Euclidean
Accuracy_Jaccard
Accuracy_Cosine

0.0775

In [43]:
print("Euclidean accuracy:",Accuracy_Euclidean)
print("Jacard accuracy:",Accuracy_Jaccard)
print("Cosine accuracy :",Accuracy_Cosine)

Euclidean accuracy: 0.07625
Jacard accuracy: 0.1075
Cosine accuracy : 0.0775
